## MLflow's Model Registry

In [1]:
from mlflow.tracking import MlflowClient


MLFLOW_TRACKING_URI = "sqlite:///mlflow.db"

### Interacting with the MLflow tracking server

The `MlflowClient` object allows us to interact with...
- an MLflow Tracking Server that creates and manages experiments and runs.
- an MLflow Registry Server that creates and manages registered models and model versions. 

To instantiate it we need to pass a tracking URI and/or a registry URI

In [7]:
client = MlflowClient(tracking_uri=MLFLOW_TRACKING_URI)

client.search_experiments()

[<Experiment: artifact_location='/workspaces/my-mlops-zoomcamp/02-experiment-tracking/mlruns/2', creation_time=1721291682907, experiment_id='2', last_update_time=1721291682907, lifecycle_stage='active', name='my-nyc-taxi-experiment', tags={}>,
 <Experiment: artifact_location='mlflow-artifacts:/0', creation_time=1721284690307, experiment_id='0', last_update_time=1721284690307, lifecycle_stage='active', name='Default', tags={}>]

In [8]:
client.create_experiment(name="my-cool-experiment")

'3'

Let's check the latest versions for the experiment with id `1`...

In [17]:
from mlflow.entities import ViewType

runs = client.search_runs(
    experiment_ids='2',
    filter_string="metrics.rmse < 10",
    run_view_type=ViewType.ACTIVE_ONLY,
    max_results=10,
    order_by=["metrics.rmse ASC"]
)

In [18]:
for run in runs:
    print(f"run id: {run.info.run_id}, rmse: {run.data.metrics['rmse']:.4f}")

run id: 5b1536a4e4ff4d51b883168f716d07dd, rmse: 6.3184
run id: 7a9ed9d2899e4b239735f60de3163650, rmse: 6.3184
run id: 0e23723387f946489fd883e531050f56, rmse: 6.3746
run id: 251e8f59205344c6a0beeafc76ca3d7d, rmse: 6.3750
run id: 40b25bb768574f05a48c7497e8c444d0, rmse: 6.4717
run id: 748c8024d7e748c79e7fa5d297d8dfa4, rmse: 6.5482
run id: 12c709c53c4a428c957139992ef5b868, rmse: 6.6007


### Interacting with the Model Registry

In this section We will use the `MlflowClient` instance to:

1. Register a new version for the experiment `nyc-taxi-regressor`
2. Retrieve the latests versions of the model `nyc-taxi-regressor` and check that a new version `4` was created.
3. Transition the version `4` to "Staging" and adding annotations to it.

In [19]:
import mlflow

mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)

## register model with client

In [20]:
run_id = "0e23723387f946489fd883e531050f56"
model_uri = f"runs:/{run_id}/model"
mlflow.register_model(model_uri=model_uri, name="nyc-taxi-regressor")

Registered model 'nyc-taxi-regressor' already exists. Creating a new version of this model...
Created version '3' of model 'nyc-taxi-regressor'.


<ModelVersion: aliases=[], creation_timestamp=1721359520563, current_stage='None', description=None, last_updated_timestamp=1721359520563, name='nyc-taxi-regressor', run_id='0e23723387f946489fd883e531050f56', run_link=None, source='/workspaces/my-mlops-zoomcamp/02-experiment-tracking/mlruns/2/0e23723387f946489fd883e531050f56/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=3>

In [33]:
model_name = "nyc-taxi-regressor"
latest_versions = client.get_model_version_by_alias(name = 'nyc-taxi-regressor', alias=)
# client.get_model_version_by_alias

# for version in latest_versions:
print(f"version: {latest_versions.version}, stage: {latest_versions.aliases}")

MlflowException: Registered model alias name cannot be empty.

## add stage or alias

In [32]:
model_version = 4
new_stage = "Staging"
client.transition_model_version_stage(
    name=model_name,
    version=model_version,
    stage=new_stage,
    archive_existing_versions=False
)

<ModelVersion: creation_timestamp=1652971637398, current_stage='Staging', description='The model version 4 was transitioned to Staging on 2022-05-19', last_updated_timestamp=1652972141519, name='nyc-taxi-regressor', run_id='b8904012c84343b5bf8ee72aa8f0f402', run_link=None, source='./mlruns/1/b8904012c84343b5bf8ee72aa8f0f402/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=4>

In [33]:
from datetime import datetime

date = datetime.today().date()
client.update_model_version(
    name=model_name,
    version=model_version,
    description=f"The model version {model_version} was transitioned to {new_stage} on {date}"
)

<ModelVersion: creation_timestamp=1652971637398, current_stage='Staging', description='The model version 4 was transitioned to Staging on 2022-05-19', last_updated_timestamp=1652972142779, name='nyc-taxi-regressor', run_id='b8904012c84343b5bf8ee72aa8f0f402', run_link=None, source='./mlruns/1/b8904012c84343b5bf8ee72aa8f0f402/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=4>

### Comparing versions and selecting the new "Production" model

In the last section, we will retrieve models registered in the model registry and compare their performance on an unseen test set. The idea is to simulate the scenario in which a deployment engineer has to interact with the model registry to decide whether to update the model version that is in production or not.

These are the steps:

1. Load the test dataset, which corresponds to the NYC Green Taxi data from the month of March 2021.
2. Download the `DictVectorizer` that was fitted using the training data and saved to MLflow as an artifact, and load it with pickle.
3. Preprocess the test set using the `DictVectorizer` so we can properly feed the regressors.
4. Make predictions on the test set using the model versions that are currently in the "Staging" and "Production" stages, and compare their performance.
5. Based on the results, update the "Production" model version accordingly.


**Note: the model registry doesn't actually deploy the model to production when you transition a model to the "Production" stage, it just assign a label to that model version. You should complement the registry with some CI/CD code that does the actual deployment.**

In [53]:
from sklearn.metrics import mean_squared_error
import pandas as pd


def read_dataframe(path_url):
    df = pd.read_parquet(path_url)

    df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)
    df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df


def preprocess(df, dv):
    df['PU_DO'] = df['PULocationID'] + '_' + df['DOLocationID']
    categorical = ['PU_DO']
    numerical = ['trip_distance']
    train_dicts = df[categorical + numerical].to_dict(orient='records')
    return dv.transform(train_dicts)


def test_model(logged_model, X_test, y_test):
    model = mlflow.pyfunc.load_model(logged_model)
    y_pred = model.predict(X_test)
    return {"rmse": mean_squared_error(y_test, y_pred, squared=False)}

In [ ]:
mlflow.pyfunc.load_model()

## download model to test prediction

In [35]:
df = read_dataframe("https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2021-03.parquet")

In [49]:
run_id = '5b1536a4e4ff4d51b883168f716d07dd'

In [37]:
client.download_artifacts(run_id=run_id, path='preprocessor', dst_path='.')

'/workspaces/my-mlops-zoomcamp/02-experiment-tracking/preprocessor'

In [38]:
import pickle

with open("preprocessor/preprocessor.b", "rb") as f_in:
    dv = pickle.load(f_in)

/home/codespace/anaconda3/envs/exp-tracking-env/lib/python3.9/site-packages/sklearn/base.py:376: InconsistentVersionWarning: Trying to unpickle estimator DictVectorizer from version 1.5.1 when using version 1.4.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [39]:
X_test = preprocess(df, dv)

In [40]:
X_test

<80372x13221 sparse matrix of type '<class 'numpy.float64'>'
	with 153356 stored elements in Compressed Sparse Row format>

In [41]:
target = "duration"
y_test = df[target].values

In [42]:
y_test

array([ 8.35      ,  5.23333333, 20.33333333, ...,  9.        ,
       25.        ,  9.        ])

In [54]:
logged_model = 'runs:/5b1536a4e4ff4d51b883168f716d07dd/models_mlflow'

In [56]:
%time test_model(logged_model=logged_model, X_test=X_test, y_test=y_test)

2024/07/19 03:53:57 WARNING mlflow.utils.requirements_utils: Detected one or more mismatches between the model's dependencies and the current Python environment:
 - scikit-learn (current: 1.4.2, required: scikit-learn==1.5.1)
To fix the mismatches, call `mlflow.pyfunc.get_model_dependencies(model_uri)` to fetch the model's environment and install dependencies using the resulting environment file.


CPU times: user 16.7 s, sys: 133 ms, total: 16.9 s
Wall time: 10.3 s


/home/codespace/anaconda3/envs/exp-tracking-env/lib/python3.9/site-packages/sklearn/metrics/_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


{'rmse': 6.2702965482607915}

In [59]:
logged_model = 'runs:/91cb2d31e60044dc8a367e9e02181ef3/models_mlflow'

In [60]:
%time test_model(logged_model=logged_model, X_test=X_test, y_test=y_test)

CPU times: user 60.6 ms, sys: 0 ns, total: 60.6 ms
Wall time: 60.1 ms


/home/codespace/anaconda3/envs/exp-tracking-env/lib/python3.9/site-packages/sklearn/metrics/_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


{'rmse': 12.181915014824462}

## set model aliases

In [63]:
client.set_registered_model_alias(model_name, 'Production', 3)

In [64]:
# client.transition_model_version_stage(
#     name=model_name,
#     version=3,
#     stage="Production",
#     archive_existing_versions=True
# )

In [65]:
print('f')

f
